<a href="https://colab.research.google.com/github/jespimentel/aposentaveis/blob/main/aposentaveis_nov_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import unicodedata
import difflib

import folium
from folium.plugins import MarkerCluster

In [2]:
membros = '/content/drive/MyDrive/Datasets/abono_novembro_24.xlsx'

# Fonte dos dados georreferenciados: https://github.com/alanwillms/geoinfo/
coordenadas_municipios = '/content/drive/MyDrive/Datasets/latitude-longitude-cidades.csv'

df_membros_ativos = pd.read_excel(membros, index_col=0)
df_coordenadas = pd.read_csv(coordenadas_municipios, sep=';', index_col=0)

In [3]:
def remover_acentos(text):
  return ''.join(c for c in unicodedata.normalize('NFD', text)
                  if unicodedata.category(c) != 'Mn')

In [4]:
def encontra_municipio(coluna):
    match coluna:
        case coluna if "PROCURADOR" in coluna:
            return 'SAO PAULO'
        case coluna if 'CAPITAL' in coluna:
            return 'SAO PAULO'
        case coluna if 'MILITAR' in coluna:
            return 'SAO PAULO'
        case coluna if 'MANDADOS' in coluna:
            return 'SAO PAULO'
        case coluna if 'HABITACAO' in coluna:
            return 'SAO PAULO'
        case coluna if 'EXECUCOES' in coluna:
          return 'SAO PAULO'
        case coluna if 'FALENCIAS' in coluna:
          return 'SAO PAULO'
        case coluna if 'HUMANOS' in coluna:
          return 'SAO PAULO'
        case coluna if 'PENHA DE FRANCA' in coluna:
          return 'SAO PAULO'
        case coluna if 'NOSSA SENHORA' in coluna:
          return 'SAO PAULO'
        case coluna if 'FORO REGIONAL' in coluna:
          return 'SAO PAULO'
        case coluna if 'PROMOTORIA DE JUSTICA REGIONAL' in coluna:
          return 'SAO PAULO'
        case coluna if 'MIMOSA' in coluna:
          return 'CAMPINAS'
        case coluna if 'IACANGA' in coluna:
          return 'IACANGA'
        case coluna if 'PROMOTORIA DE JUSTICA DE DISTRITAL DE ' in coluna:
          return coluna.split('PROMOTORIA DE JUSTICA DE DISTRITAL DE ')[-1]
        case coluna if 'PROMOTORIA DE JUSTICA DE ' in coluna:
          return coluna.split('PROMOTORIA DE JUSTICA DE ')[-1]
        case coluna if 'PROMOTORIAS DE JUSTICA DE AREA REGIONAL DE ' in coluna:
          return coluna.split('PROMOTORIAS DE JUSTICA DE AREA REGIONAL DE ')[-1]
        case coluna if 'PROMOTORIAS DE JUSTICA DA AREA REGIONAL DE ' in coluna:
          return coluna.split('PROMOTORIAS DE JUSTICA DA AREA REGIONAL DE ')[-1]
        case coluna if 'PROMOTORIA DE JUSTICA DISTRITAL DE ' in coluna:
          return coluna.split('PROMOTORIA DE JUSTICA DISTRITAL DE ')[-1]

        case _:
            return 'SAO PAULO'

In [5]:
def encontra_semelhante(texto, lista_referencia):
  try:
    return difflib.get_close_matches(texto, lista_referencia, 1, 0.3)[0]
  except IndexError:
    return 'NAO ENCONTRADO'

In [6]:
def gera_mapa_clusterizado(df, cores_dict):
  COORDENADAS_MEDIAS = df['latitude'].mean(), df['longitude'].mean()
  # Cria o mapa vazio
  mapa = folium.Map(location=COORDENADAS_MEDIAS, zoom_start=7)

  # Cria um MarkerCluster para agrupar os marcadores
  marker_cluster = MarkerCluster().add_to(mapa)

  # Cria um dicionário de cores para os marcadores
  cores = cores_dict

  # Cor padrão para naturezas não especificadas
  cor_padrao = 'beige'

  # Adiciona os marcadores
  for _, row in df.iterrows():

    html_popup = f"""
    <strong>{row['Cargo']}</strong> <br>
    {row['Lotação']}<br>
    """
    # Determina a cor do marcador com base na natureza
    cor = cores.get(row['Cargo'], cor_padrao)

    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=html_popup,
        icon=folium.Icon(color=cor, icon='info-sign')
    ).add_to(marker_cluster)

  # Adiciona o título ao mapa
  titulo_html = '''
              <h3 align="center" style="font-size:20px"><b>Onde estão os colegas que já podem se aposentar? (ref.: Nov. 2024)</b></h3>
              '''
  mapa.get_root().html.add_child(folium.Element(titulo_html))

  # Adiciona o rodapé ao mapa
  rodape_html = '''
              <p align="center" style="font-size:12px;color:gray;"><i>github.com/jespimentel/aposentaveis</i></p>
              '''
  mapa.get_root().html.add_child(folium.Element(rodape_html))

  return mapa

In [7]:
# Adequação do dataframe de membros ativos
df_membros_ativos['Lotação'] = df_membros_ativos['Lotação'].str.upper()
df_membros_ativos['Lotação'] = df_membros_ativos['Lotação'].apply(remover_acentos)
df_membros_ativos['Município'] = df_membros_ativos['Lotação'].apply(encontra_municipio)

df_membros_ativos[['Cargo', 'Lotação', 'Município']].sample(5)

,Cargo,Lotação,Município
Matrícula,,,
201151,PROMOTOR DE JUSTICA (ENTRANCIA FINAL),PROMOTORIA DE JUSTICA REGIONAL DE SAO MIGUEL P...,SAO PAULO
7521,PROMOTOR DE JUSTICA (ENTRANCIA INTERMEDIARIA),PROMOTORIA DE JUSTICA DISTRITAL DE CAMPO LIMPO...,CAMPO LIMPO PAULISTA
495168,PROCURADOR DE JUSTICA,PROCURADORIA DE JUSTICA CRIMINAL,SAO PAULO
3350,PROMOTOR DE JUSTICA (ENTRANCIA FINAL),PROMOTORIA DE JUSTICA DE PRAIA GRANDE,PRAIA GRANDE
534178,PROCURADOR DE JUSTICA,PROCURADORIA DE JUSTICA CRIMINAL,SAO PAULO


In [8]:
# Cria o dataframe com as coordenadas dos municípios do Estado de SP
df_coordenadas_sp = df_coordenadas.loc[df_coordenadas['uf'] == 'SP'].copy()

# Adequação do dataframe de coordenadas
df_coordenadas_sp['municipio'] = df_coordenadas_sp['municipio'].str.upper()
df_coordenadas_sp['municipio'] = df_coordenadas_sp['municipio'].apply(remover_acentos)
df_coordenadas_sp = df_coordenadas_sp.drop(columns=['uf'])
df_coordenadas_sp.sample(5)

,municipio,longitude,latitude
id_municipio,,,
4909,EUCLIDES DA CUNHA PAULISTA,-52.592779,-22.554520
4970,INDIANA,-51.255467,-22.173779
5165,PIQUETE,-45.186919,-22.606881
4823,BURI,-48.595811,-23.797689
5012,JACI,-49.579732,-20.880468


In [9]:
lista_referencia = df_coordenadas_sp['municipio'].to_list()
lista_referencia = list(set(lista_referencia))

df_membros_ativos['municipio'] = df_membros_ativos['Município'].apply(lambda x: encontra_semelhante(x, lista_referencia))

In [10]:
# Cria o dataframe de membros com requisitos para se aposentar
df_aposentaveis = df_membros_ativos.loc[df_membros_ativos['Abono'] != ' - '].drop(columns=['Abono'])

In [11]:
print('\nPodem se aposentar:')
display(df_aposentaveis.groupby('Cargo')['Município'].count().sort_values(ascending=False))


Podem se aposentar:


,Município
Cargo,
PROCURADOR DE JUSTICA,235
PROMOTOR DE JUSTICA (ENTRANCIA FINAL),186
PROMOTOR DE JUSTICA (ENTRANCIA INTERMEDIARIA),9
PROMOTOR DE JUSTICA (ENTRANCIA INICIAL),2
PROCURADOR GERAL DE JUSTICA,1


In [12]:
# Fazendo o merge dos DataFrames
df_merged = pd.merge(df_aposentaveis, df_coordenadas_sp, on='municipio')
df_merged = df_merged[['Cargo', 'Lotação', 'Município', 'latitude', 'longitude']]

display(df_merged)

,Cargo,Lotação,Município,latitude,longitude
0,PROCURADOR DE JUSTICA,PROCURADORIA DE JUSTICA CRIMINAL,SAO PAULO,-23.532905,-46.639520
1,PROMOTOR DE JUSTICA (ENTRANCIA FINAL),PROMOTORIA DE JUSTICA DE OURINHOS,OURINHOS,-22.979695,-49.869702
2,PROCURADOR DE JUSTICA,PROCURADORIA JUSTICA HABEAS CORPUS MANDADOS SE...,SAO PAULO,-23.532905,-46.639520
3,PROMOTOR DE JUSTICA (ENTRANCIA FINAL),PROMOTORIA DE JUSTICA REGIONAL DE SANTANA,SAO PAULO,-23.532905,-46.639520
4,PROMOTOR DE JUSTICA (ENTRANCIA FINAL),PROMOTORIA DE JUSTICA DA CAPITAL,SAO PAULO,-23.532905,-46.639520
...,...,...,...,...,...
428,PROMOTOR DE JUSTICA (ENTRANCIA FINAL),PROMOTORIA DE JUSTICA DE OSASCO,OSASCO,-23.532390,-46.791555
429,PROMOTOR DE JUSTICA (ENTRANCIA FINAL),PROMOTORIA DE JUSTICA DE JALES,JALES,-20.267227,-50.549442
430,PROMOTOR DE JUSTICA (ENTRANCIA FINAL),PROMOTORIA DE JUSTICA DA CAPITAL,SAO PAULO,-23.532905,-46.639520
431,PROCURADOR DE JUSTICA,PROCURADORIA DE JUSTICA CRIMINAL,SAO PAULO,-23.532905,-46.639520


In [13]:
display(df_merged.groupby(['Município', 'Cargo'])['Lotação'].count().sort_values(ascending=False))

Município              Cargo                                        
SAO PAULO              PROCURADOR DE JUSTICA                            235
                       PROMOTOR DE JUSTICA (ENTRANCIA FINAL)             68
CAMPINAS               PROMOTOR DE JUSTICA (ENTRANCIA FINAL)             13
RIBEIRAO PRETO         PROMOTOR DE JUSTICA (ENTRANCIA FINAL)             10
BAURU                  PROMOTOR DE JUSTICA (ENTRANCIA FINAL)              8
MARILIA                PROMOTOR DE JUSTICA (ENTRANCIA FINAL)              6
OSASCO                 PROMOTOR DE JUSTICA (ENTRANCIA FINAL)              6
SANTOS                 PROMOTOR DE JUSTICA (ENTRANCIA FINAL)              6
SOROCABA               PROMOTOR DE JUSTICA (ENTRANCIA FINAL)              5
SAO JOSE DO RIO PRETO  PROMOTOR DE JUSTICA (ENTRANCIA FINAL)              5
SAO JOSE DOS CAMPOS    PROMOTOR DE JUSTICA (ENTRANCIA FINAL)              4
PRESIDENTE PRUDENTE    PROMOTOR DE JUSTICA (ENTRANCIA FINAL)              4
FRANCA                 PROMOTOR DE JUSTICA (ENTRANCIA FINAL)              4
SANTO ANDRE            PROMOTOR DE JUSTICA (ENTRANCIA FINAL)              3
SAO BERNARDO DO CAMPO  PROMOTOR DE JUSTICA (ENTRANCIA FINAL)              3
ARACATUBA              PROMOTOR DE JUSTICA (ENTRANCIA FINAL)              3
BOTUCATU               PROMOTOR DE JUSTICA (ENTRANCIA FINAL)              3
AMERICANA              PROMOTOR DE JUSTICA (ENTRANCIA FINAL)              2
SAO CARLOS             PROMOTOR DE JUSTICA (ENTRANCIA FINAL)              2
SAO CAETANO DO SUL     PROMOTOR DE JUSTICA (ENTRANCIA FINAL)              2
MOGI DAS CRUZES        PROMOTOR DE JUSTICA (ENTRANCIA FINAL)              2
MIRASSOL               PROMOTOR DE JUSTICA (ENTRANCIA INTERMEDIARIA)      2
ITAPETININGA           PROMOTOR DE JUSTICA (ENTRANCIA FINAL)              2
JUNDIAI                PROMOTOR DE JUSTICA (ENTRANCIA FINAL)              2
JAU                    PROMOTOR DE JUSTICA (ENTRANCIA FINAL)              2
GUARULHOS              PROMOTOR DE JUSTICA (ENTRANCIA FINAL)              2
CATANDUVA              PROMOTOR DE JUSTICA (ENTRANCIA FINAL)              2
BARUERI                PROMOTOR DE JUSTICA (ENTRANCIA FINAL)              2
SAO PAULO              PROCURADOR GERAL DE JUSTICA                        1
SAO JOSE DO RIO PARDO  PROMOTOR DE JUSTICA (ENTRANCIA INTERMEDIARIA)      1
SAO JOAO DA BOA VISTA  PROMOTOR DE JUSTICA (ENTRANCIA FINAL)              1
SAO VICENTE            PROMOTOR DE JUSTICA (ENTRANCIA FINAL)              1
TAUBATE                PROMOTOR DE JUSTICA (ENTRANCIA FINAL)              1
VALINHOS               PROMOTOR DE JUSTICA (ENTRANCIA FINAL)              1
VINHEDO                PROMOTOR DE JUSTICA (ENTRANCIA INTERMEDIARIA)      1
ADAMANTINA             PROMOTOR DE JUSTICA (ENTRANCIA INICIAL)            1
MOGI-GUACU             PROMOTOR DE JUSTICA (ENTRANCIA FINAL)              1
RIO CLARO              PROMOTOR DE JUSTICA (ENTRANCIA FINAL)              1
RIBEIRAO PRETO         PROMOTOR DE JUSTICA (ENTRANCIA INTERMEDIARIA)      1
PIRACICABA             PROMOTOR DE JUSTICA (ENTRANCIA FINAL)              1
OURINHOS               PROMOTOR DE JUSTICA (ENTRANCIA FINAL)              1
OSVALDO CRUZ           PROMOTOR DE JUSTICA (ENTRANCIA INTERMEDIARIA)      1
LINS                   PROMOTOR DE JUSTICA (ENTRANCIA FINAL)              1
JALES                  PROMOTOR DE JUSTICA (ENTRANCIA FINAL)              1
GUARATINGUETA          PROMOTOR DE JUSTICA (ENTRANCIA FINAL)              1
CAPIVARI               PROMOTOR DE JUSTICA (ENTRANCIA INTERMEDIARIA)      1
BILAC                  PROMOTOR DE JUSTICA (ENTRANCIA INICIAL)            1
BATATAIS               PROMOTOR DE JUSTICA (ENTRANCIA INTERMEDIARIA)      1
AVARE                  PROMOTOR DE JUSTICA (ENTRANCIA FINAL)              1
ASSIS                  PROMOTOR DE JUSTICA (ENTRANCIA FINAL)              1
ARARAS                 PROMOTOR DE JUSTICA (ENTRANCIA INTERMEDIARIA)      1
ARARAQUARA             PROMOTOR DE JUSTICA (ENTRANCIA F

In [14]:
cores_dict = {
      'PROCURADOR DE JUSTICA': 'black',
      'PROCURADOR GERAL DE JUSTICA': 'black',
      'PROMOTOR DE JUSTICA (ENTRANCIA FINAL)': 'darkred',
      'PROMOTOR DE JUSTICA (ENTRANCIA INTERMEDIARIA)': 'blue',
      'PROMOTOR DE JUSTICA (ENTRANCIA INICIAL)': 'green'
  }

In [15]:
# Gerando o mapa clusterizado
mapa = gera_mapa_clusterizado(df_merged, cores_dict)
mapa.save('index.html')
display(mapa)